##### Generating CSV Data and DF for Pedido and Item_Pedido

In [81]:

import csv
import pandas as pd
from random import shuffle, choice 

id_unidade = [id for id in range(1, 8)]
vlr_pedido = [111, 122, 225, 1008, 409, 888, 77]
vlr_item = [2.87, 88.77, 77.65, 10.13, 14.8, 9.88, 1.22]
shuffle(id_unidade)
shuffle(vlr_pedido)
shuffle(vlr_item)
pedido = {
        "Id_Unidade": id_unidade, 
        "Id_Pedido":  [id for id in range(1, 8)], 
        "Tipo_Pedido": [choice((1, 2)) for tipo_p in range(1, 8)],
        "Data_Pedido": [None for data_p in range(1, 8)],
        "Vlr_Pedido": vlr_pedido,
        "Endereco_Entrega": [None for endereco in range(1, 8)],
        "Taxa_Entrega": [None for taxa_entrega in range(1, 8)],
        "Status": [choice(("Pendente", "Finalizado", "Cancelado")) for st in range(1, 8)]
        }
id_pedido = id_unidade.copy()
shuffle(id_pedido)
id_produto = id_pedido.copy()
shuffle(id_produto)
item_pedido = { 
        "Id_Pedido":  id_pedido, 
        "Id_Item_Pedido": [id for id in range(1, 8)],
        "Id_Produto": id_produto,
        "Qtd": [choice([i for i in range(100)]) for qtd in range(1, 8)],
        "Vlr_Item": vlr_item,
        "Observacao": [None for taxa_entrega in range(1, 8)],
        }

df_pedido = pd.DataFrame(pedido)
df_item_pedido = pd.DataFrame(item_pedido)

print("Pedido DF: ")
display(df_pedido)
print("Item Pedido DF: ")
display(df_item_pedido)


Pedido DF: 


,Id_Unidade,Id_Pedido,Tipo_Pedido,Data_Pedido,Vlr_Pedido,Endereco_Entrega,Taxa_Entrega,Status
0,6,1,1,None,888,None,None,Cancelado
1,1,2,1,None,111,None,None,Cancelado
2,2,3,1,None,1008,None,None,Pendente
3,5,4,2,None,409,None,None,Finalizado
4,3,5,2,None,122,None,None,Finalizado
5,4,6,2,None,77,None,None,Cancelado
6,7,7,1,None,225,None,None,Cancelado


Item Pedido DF: 


,Id_Pedido,Id_Item_Pedido,Id_Produto,Qtd,Vlr_Item,Observacao
0,3,1,3,44,10.13,None
1,7,2,5,12,14.80,None
2,1,3,4,17,9.88,None
3,4,4,2,83,77.65,None
4,2,5,6,3,1.22,None
5,5,6,1,98,2.87,None
6,6,7,7,9,88.77,None


##### Creation of CSVs

In [98]:
def pivot_table(table_values):
    values, temp_list = [], []
    for i in range(len(list(table_values.values())[0])):
        for value in table_values.values():
            temp_list += [value[i]]
        values.append(temp_list)
        temp_list = []
    return values

pedido_values, item_pedido_values = pivot_table(pedido), pivot_table(item_pedido)

def create_csv(name, table, table_values):
    with open(f"{name}.csv", 'w') as file:
        writer = csv.writer(file)
        writer.writerow(table)
    
        for row in table_values:
            writer.writerow(row)

create_csv("pedido", pedido, pedido_values)
create_csv("item_pedido", item_pedido, item_pedido_values)